In [52]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
!docker run -d -p 6333:6333 -p 6334:6334 \
   -v "./qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


49c3633ed9c20df335aa88239b8d96715ee0944dab26b708d72da8b6ac606495


# Q1. Embedding the query

In [1]:
from fastembed import TextEmbedding

In [2]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
q = 'I just discovered the course. Can I join now?'

In [4]:
# Initialize the model
model = TextEmbedding(model_name=model_handle)

# Generate embedding (returns a generator, so convert to list first)
embedding_vector_q = list(model.embed([q]))[0]
print(embedding_vector_q.min())


-0.11726373885183883


In [6]:
embedding_vector_q.dot(embedding_vector_q)

np.float64(1.0000000000000002)

# Q2. Cosine similarity with another vector

In [7]:
doc = 'Can I still join the course after the start date?'

In [8]:
# Initialize the model
model = TextEmbedding(model_name=model_handle)

# Generate embedding (returns a generator, so convert to list first)
embedding_vector_doc = list(model.embed([doc]))[0]

In [9]:
import numpy as np

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

similarity = cosine_similarity(embedding_vector_q, embedding_vector_doc)
print(f"Cosine similarity: {similarity}")

Cosine similarity: 0.9008528895674548


# Q3 Ranking by cosine

In [10]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [11]:
documents[0]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
cos_sim = []
for doc in documents:
    model = TextEmbedding(model_name=model_handle)
    embedding_vector_doc = list(model.embed([doc['text']]))[0]
    similarity = cosine_similarity(embedding_vector_q, embedding_vector_doc)
    cos_sim.append(similarity)
cos_sim

[np.float64(0.7629684696540238),
 np.float64(0.8182378150042889),
 np.float64(0.8085397398734189),
 np.float64(0.7133079015686243),
 np.float64(0.7304499234333598)]

# Q4. Ranking by cosine, version two

In [13]:
full_text = doc['question'] + ' ' + doc['text']

In [14]:
cos_sim_v2 = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    model = TextEmbedding(model_name=model_handle)
    embedding_vector_doc = list(model.embed([full_text]))[0]
    similarity = cosine_similarity(embedding_vector_q, embedding_vector_doc)
    cos_sim_v2.append(similarity)
cos_sim_v2

[np.float64(0.8514543236908069),
 np.float64(0.8436594159113068),
 np.float64(0.840828704850256),
 np.float64(0.7755157969663908),
 np.float64(0.8086007795043938)]

# Q5. Selecting the embedding model

In [72]:
models = TextEmbedding.list_supported_models()
models

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [27]:
min_dim = float('inf')

for model in models:
    if model['dim'] <= min_dim:
        min_dim = model['dim']
min_dim

384

# Q6. Indexing with qdrant (2 points)

In [54]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [55]:
documents[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp'}

In [56]:
from qdrant_client import QdrantClient, models

In [57]:
qd_client = QdrantClient("http://localhost:6333")

In [58]:
model_handle = 'BAAI/bge-small-en-v1.5'
EMBEDDING_DIMENSIONALITY = 384

# Define the collection name
collection_name = "homework_2"

In [59]:
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE
    )
)

True

In [60]:
points = []
id = 0

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)


In [63]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [67]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [68]:
result = search(q)

In [69]:
result

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.7397805, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])